# Wind Speed Prediction

## Imports

In [4]:
import pandas as pd
from tabulate import tabulate

## Dataset

In [3]:
df = pd.read_csv('wind_dataset.csv', parse_dates=True, index_col=0)
df.head()

,WIND,IND,RAIN,IND.1,T.MAX,IND.2,T.MIN,T.MIN.G
DATE,,,,,,,,
1961-01-01,13.67,0,0.2,0.0,9.5,0.0,3.7,-1.0
1961-01-02,11.50,0,5.1,0.0,7.2,0.0,4.2,1.1
1961-01-03,11.25,0,0.4,0.0,5.5,0.0,0.5,-0.5
1961-01-04,8.63,0,0.2,0.0,5.6,0.0,0.4,-3.2
1961-01-05,11.92,0,10.4,0.0,7.2,1.0,-1.5,-7.5


In [10]:
summary = df.describe().transpose().round(2)
summary = summary.drop('count', axis=1)
table = tabulate(summary, headers='keys', tablefmt='rounded_grid')
print(table)

╭─────────┬────────┬───────┬───────┬───────┬───────┬───────┬───────╮
│         │   mean │   std │   min │   25% │   50% │   75% │   max │
├─────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ WIND    │   9.8  │  4.98 │   0   │   6   │  9.21 │ 12.96 │ 30.37 │
├─────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ IND     │   0.39 │  1.18 │   0   │   0   │  0    │  0    │  4    │
├─────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ RAIN    │   1.89 │  4.03 │   0   │   0   │  0.2  │  2    │ 67    │
├─────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ IND.1   │   0.36 │  1.13 │   0   │   0   │  0    │  0    │  4    │
├─────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ T.MAX   │  13.34 │  4.89 │  -0.1 │   9.6 │ 13.3  │ 17.2  │ 26.8  │
├─────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ IND.2   │   0.46 │  1.18 │   0   │   0   │  0    │  0    │  4    │
├─────────┼────────┼───────┼──────